In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 847 kB/s eta 0:00:01
     |████████████████████████████████| 981 kB 54.4 MB/s eta 0:00:01
     |████████████████████████████████| 263 kB 60.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 49.8 MB/s eta 0:00:01
     |████████████████████████████████| 468 kB 56.4 MB/s eta 0:00:01
  Created wheel for ktrain: filename=ktrain-0.27.2-py3-none-any.whl size=25283078 sha256=5e4b95aae76c2451d97f5e03c6f42ab73860a1560ee3b68b3359e3c35fdaa1c0
  Stored in directory: /root/.cache/pip/wheels/88/be/4a/971c83a380a40f12e877f643ca1b94dc65f528f94c88dbcff7
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9918 sha256=62b776e7d078bc4131eadd417a5bc69a6513eba93e0c7c9ba09595af55f8f284
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff05c722d142b7d20a88ad63f9aab11b895411241a4
  Created wheel for keras-bert: filename=keras_bert-0.88.0-py3-none-any.whl size=34205 sha256=c01f7c10e1aba0afb7b344ae40372f62d16c6b9aa

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import ktrain
from ktrain import text

In [4]:
data = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
X= data['review']
y=data['sentiment']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5)

In [6]:
MODEL_NAME = 'albert-base-v2'
t = text.Transformer(MODEL_NAME, maxlen=400, classes=np.array(['positive','negative']))
trn = t.preprocess_train(np.array(X_train),np.array(y_train))
val = t.preprocess_test(np.array(X_test), np.array(y_test))
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/opt/conda/lib/python3.7/site-packages/ktrain/text/preprocessor.py:420: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 231
	95percentile : 586
	99percentile : 903


/opt/conda/lib/python3.7/site-packages/ktrain/utils.py:600: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 231
	95percentile : 595
	99percentile : 910


Downloading:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

In [7]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 1953s 464ms/step - loss: 0.3347 - accuracy: 0.8543 - val_loss: 0.1723 - val_accuracy: 0.9352


In [8]:
predictor = ktrain.get_predictor(learner.model,preproc = t)


In [9]:
predictor.predict("A delightful diversion from the MCU as we know it, Loki successfully sees star Tom Hiddleston leap from beloved villain to endearing antihero -- with a little help from Owen Wilson -- in a series that's as off-kilter, charming, and vaguely dangerous as the demigod himself.")

'positive'

In [10]:
predictor.predict("Wouldn't watch it again.")

'negative'

In [11]:
predictor.save('./albert')